# Graphviz

library with tutorial: https://pypi.org/project/graphviz/

python doc: https://www.graphviz.org/pdf/gv.3python.pdf

another doc: https://github.com/xflr6/graphviz

paper: https://www.graphviz.org/pdf/dotguide.pdf


In [12]:
from graphviz import Digraph
import re
import os
import json

## an example

In [3]:
dot = Digraph('testing')

In [4]:
dot.node('sf__opportunity_products', 'sf__opportunity_products')
dot.edge('sf__opportunities', 'sf__opportunities_extended')
dot.edge('sf__opportunities_extended', 'sf__accounts', label='${sf__opportunities_extended.account_id} = ${sf__accounts.id}')
dot.edge('sf__accounts', 'sf__users', label='${sf__accounts.owner_id} = ${sf__users.id}')
print(dot.source)


digraph testing {
	sf__opportunity_products [label=sf__opportunity_products]
	sf__opportunities -> sf__opportunities_extended
	sf__opportunities_extended -> sf__accounts [label="${sf__opportunities_extended.account_id} = ${sf__accounts.id}"]
	sf__accounts -> sf__users [label="${sf__accounts.owner_id} = ${sf__users.id}"]
}


In [5]:
dot.render('sf_model.gv', view=True)

'sf_model.gv.pdf'

## getting real

In [332]:
with open('explores/etl_jobs/status.json', 'r') as f:
    model = json.load(f)

In [333]:
s = model['explore']

In [98]:
for n, l in enumerate(s): print(n,l)

0 explore: sf__campaign_members {
1   label: "Campaign Members"
2   join: sf__campaigns {
3 
4     sql_on: ${sf__campaign_members.campaign_id} = ${sf__campaigns.id} ;;
5     relationship: many_to_one
6   }
7 
8   join: sf__leads {
9     sql_on: ${sf__campaign_members.lead_id} = ${sf__leads.id} ;;
10     relationship: many_to_one
11   }
12 
13   join: sf__contacts {
14     sql_on: ${sf__campaign_members.lead_or_contact_id} = ${sf__contacts.id} ;;
15     relationship: many_to_one
16   }
17 
18   join: docker_users {
19     from: reghub_dockeruser
20     sql_on: docker_users.email = sf__campaign_members.email ;;
21     relationship: many_to_one
22   }
23 
24   join: ucp_licenses {
25     from: ucp_licensing
26     sql_on: ucp_licenses.hub_uuid = replace(${docker_users.uuid}, '-','') ;;
27     required_joins: [docker_users]
28     relationship: many_to_one
29   }
30 
31   join: ucp_usage {
32     sql_on: ucp_usage.license_id = ucp_licenses.license_key ;;
33     required_joins: [ucp_license

In [179]:
num0 = 0
counter = num0+1
counter_list = []
while counter < len(s): 
    num0 = counter
    counter = num0 + 1
    num1 = next((num for num, line in enumerate(s[counter:]) if 'join:' in line), len(s))

    if num1 != len(s):
        counter += num1
        counter_list.append(counter)
    else:
        counter = len(s)
    
    
    

In [180]:
counter_list

[2, 8, 13, 18, 24, 31]

In [229]:
grouped_explore = []
grouped_explore.append(s[:counter_list[0]])
for r in range(len(counter_list)-1):
    grouped_explore.append(s[counter_list[r]: counter_list[r+1]])
grouped_explore.append(s[counter_list[-1]:])

In [230]:
grouped_explore

[['explore: sf__campaign_members {', '  label: "Campaign Members"'],
 ['  join: sf__campaigns {',
  '',
  '    sql_on: ${sf__campaign_members.campaign_id} = ${sf__campaigns.id} ;;',
  '    relationship: many_to_one',
  '  }',
  ''],
 ['  join: sf__leads {',
  '    sql_on: ${sf__campaign_members.lead_id} = ${sf__leads.id} ;;',
  '    relationship: many_to_one',
  '  }',
  ''],
 ['  join: sf__contacts {',
  '    sql_on: ${sf__campaign_members.lead_or_contact_id} = ${sf__contacts.id} ;;',
  '    relationship: many_to_one',
  '  }',
  ''],
 ['  join: docker_users {',
  '    from: reghub_dockeruser',
  '    sql_on: docker_users.email = sf__campaign_members.email ;;',
  '    relationship: many_to_one',
  '  }',
  ''],
 ['  join: ucp_licenses {',
  '    from: ucp_licensing',
  "    sql_on: ucp_licenses.hub_uuid = replace(${docker_users.uuid}, '-','') ;;",
  '    required_joins: [docker_users]',
  '    relationship: many_to_one',
  '  }',
  ''],
 ['  join: ucp_usage {',
  '    sql_on: ucp_usag

In [308]:
grouped_explore[0]

['explore: sf__campaign_members {',
 '  label: "Campaign Members"',
 'from: base_explore']

In [302]:
explore_tree = dict()

explore_base = list(filter(None, grouped_explore[0][0].split(' ')))[1]
for line in grouped_explore[0][1:]:
    if 'from:' in line:
        explore_base = list(filter(None, line.split(' ')))[1]
explore_tree['explore_base'] = explore_base

joins = []
for clause in grouped_explore[1:]:
    join_name = list(filter(None, clause[0].split(' ')))[1]
    
    for line in clause[1:]:
        if 'from:' in line:
            join_name = list(filter(None, line.split(' ')))[1]
    joins.append(join_name)
explore_tree['joins'] = joins

In [303]:
explore_tree

{'explore_base': 'base_explore',
 'joins': ['sf__campaigns',
  'sf__leads',
  'sf__contacts',
  'reghub_dockeruser',
  'ucp_licensing',
  'ucp_usage']}

In [320]:
def trace_tree(grouped_explore):
    explore_tree = dict()

    explore_tree['explore_base'] = trace_base(grouped_explore[0])

    joins = []
    for clause in grouped_explore[1:]:
        joins.append(trace_base(clause))

    explore_tree['joins'] = joins
    return explore_tree

In [321]:
explore_tree = trace_tree(grouped_explore)

In [316]:
def trace_base(clause):
    for line in clause[1:]:
        base = list(filter(None, clause[0].split(' ')))[1]
        if 'from:' in line:
            base = list(filter(None, line.split(' ')))[1]
    return base    

In [322]:
explore_tree

{'explore_base': 'base_explore',
 'joins': ['sf__campaigns',
  'sf__leads',
  'sf__contacts',
  'docker_users',
  'ucp_licenses',
  'ucp_usage']}

# testing main

In [362]:
import re
import os
import json
import logging


def divider(explore_list):
    """
    This function returns a list of location number that points to the joins in the explore list.

    :param explore_list: the list of explore, each element is a line from the explore, with possible joins.

    :type explore_list: list

    :return: a list of integers
    """
    num0 = 0
    counter = num0+1
    counter_list = []
    while counter < len(explore_list): 
        num0 = counter
        counter = num0 + 1
        num1 = next((num for num, line in enumerate(explore_list[counter:]) if 'join:' in line), len(explore_list))

        if num1 != len(explore_list):
            counter += num1
            counter_list.append(counter)
        else:
            counter = len(explore_list)

    return counter_list


def parser(explore_list, loc_list):
    """
    This function parses a list of explore lines into a grouped structure of explore lines.

    :param explore_list: the list representing raw explore file.

    :type explore_list: list

    :param loc_list: the list of dividers, each divider is the number of join in the explore list

    :type loc_list: list

    :return: a grouped and nested list representing the explore structure with joins.
    """
    grouped_explore = []
    grouped_explore.append(explore_list[:loc_list[0]])
    for r in range(len(loc_list)-1):
        grouped_explore.append(explore_list[loc_list[r]: loc_list[r+1]])
    grouped_explore.append(explore_list[loc_list[-1]:])
    return grouped_explore


def trace_base(clause):
    """
    This function traces down a join or explore clause back to the base view name.

    :param clause: a list of one Lookml clause, that is either an explore level or join level.

    :type clause: list

    :return: the base view name. string type
    """
    if len(clause) > 1:
        for line in clause[1:]:
            base = list(filter(None, clause[0].split(' ')))[1]
            if 'from:' in line:
                base = list(filter(None, line.split(' ')))[1]
    else:
        base = list(filter(None, clause[0].split(' ')))[1]
    return base   


def trace_joins(grouped_explore):
    """
    This function generates a dictionary of the explore tree.

    :param grouped_explore: a list representing one explore, with each element being either the explore base view details, or the joined view details.

    :type grouped_explore: list

    :return: a list representing all the joined base view names.
    """
    joins = []
    for clause in grouped_explore:
        joins.append(trace_base(clause))

    return joins

In [363]:
with open(f'explores/salesforce/sf__cases.json', 'r') as f:
    model = json.load(f)

explore_name = list(filter(None, model['explore'][0].split(' ')))[1]
explore_list = model['explore']
# logging.info(model_name, explore_name)

# find the divider of explore level and join level clauses
loc_list = divider(explore_list)

if len(loc_list) > 1:
    # parse the raw list, generate a nested and well grouped list representing the explore and join structure
    grouped_explore = parser(explore_list, loc_list)
    # generate a list of all joined base view names
    explore_joins = trace_joins(grouped_explore)
else: 
    explore_joins = trace_joins([explore_list])

explore_dict = dict()
explore_dict['explore_name'] = explore_name
explore_dict['explore_joins'] = explore_joins

In [364]:
trace_joins(grouped_explore)

['sf__cases',
 'sf__case_comments',
 'sf__accounts',
 'sf__opportunities',
 'opportunity_products',
 'sf__opportunity_revenue_movements',
 'sf__opportunity_total_revenue',
 'sf__contacts',
 'owners',
 'creators',
 'sf__leads']

In [367]:
grouped_explore

[['explore: sf__cases {',
  '  persist_for: "24 hours"',
  '  label: "Cases"',
  '  # Label the view in the Explore UI as Cases (instead of the long explore name)',
  '  view_label: "Cases"',
  '  # Add a convenient description to Explore',
  '  description: "Cases and associated comments"',
  ''],
 ['  join: sf__case_comments {',
  '    view_label: "Case Comments"',
  '    sql_on: ${sf__cases.id} = ${sf__case_comments.parent_id} ;;',
  '    relationship: one_to_many',
  '  }',
  ''],
 ['  join: sf__accounts {',
  '    view_label: "Case Accounts"',
  '    sql_on: ${sf__cases.account_id} = ${sf__accounts.id} ;;',
  '    relationship: many_to_one',
  '  }',
  ''],
 ['  join: sf__opportunities {',
  '    view_label: "Opportunities"',
  '    sql_on: ${sf__opportunities.account_id} = ${sf__accounts.id} ;;',
  '    relationship: one_to_many',
  '  }',
  ''],
 ['  join: opportunity_products {',
  '    from: sf__opportunity_products',
  '    view_label: "Opportunitiy Products"',
  '    sql_on:

In [365]:
joins = []
for clause in grouped_explore:
    joins.append(trace_base(clause))

In [378]:
grouped_explore[4]

['  join: opportunity_products {',
 '    from: sf__opportunity_products',
 '    view_label: "Opportunitiy Products"',
 '    sql_on: ${sf__opportunities.id} = ${opportunity_products.opportunity_id} ;;',
 '    relationship: one_to_many',
 '  }',
 '']

In [389]:
for line in grouped_explore[4][1:]:

    base = list(filter(None, grouped_explore[4][0].split(' ')))[1]
    
    if 'from:' in line:
        print('yes')
        base = list(filter(None, line.split(' ')))[1]
        break
        

yes


In [386]:
line = grouped_explore[4][1]

In [381]:
list(filter(None, line.split(' ')))[1]

'sf__opportunity_products'

In [388]:
'from:' in line

True

In [387]:
line

'    from: sf__opportunity_products'